In [6]:
# Global toolbox
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pandas_datareader import DataReader
from datetime import datetime
import scipy.stats as ss
import yfinance as yf



In [8]:
# Data initialization
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 8, 31)
stock_symbol = 'NVDA'
stocks= yf.download(stock_symbol, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [5]:
stocks['Adj Close'].describe() # summary statistics
X = stocks['Adj Close'].values
print(f"sample mean: ${np.mean(X)}")
print(f"sample standard deviation: ${np.std(X)}")
print("sample skewness: ",ss.skew(X, axis=0, bias=True))
print("sample kurtosis: ",ss.kurtosis(X, axis=0, bias=True)+3)
# Empirical Distribution
sns.kdeplot(data=X,linewidth=4)

sample mean: $31.725553992659716
sample standard deviation: $29.64219549687437


NameError: name 'ss' is not defined